<div>lookups</div><div>-----------<br></div>

In [1]:
import org.apache.spark.rdd._
import org.apache.spark.sql._
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions._
import group.research.aging.spark.extensions._

In [2]:
import ammonite.ops._
import ammonite.ops.ImplicitWd._

<div><b>Making a list of species</b></div><div>-------------------------------<br></div>

In [4]:
//val anage = spark.readTSV("/data/databases/anage/anage_data.tsv", header = true).withColumn("scientific_name", concat($"Genus", lit(" "), $"Species"))
//anage.show()
//val anage_animals = spark.readTSV("/data/ensembl/99/view_animals_anage.tsv", header = true).withColumnRenamed("latin_name", "scientific_name")
//anage_animals.show(10,10000)
val species = spark.readTSV("/data/ensembl/99/ensembl_anage_vertebrates.tsv", header = true).withColumnRenamed("latin_name", "scientific_name")
species.show(10,10000)

+--------------------+--------------------------------+-----------+-----------+------------------+-----------------------+-----+-----+------+-------+----------------+-----------+--------------+-----------+--------------+--------------------+------------------+-------+-----------+----------------+--------------------+--------------+----------------+--------------+-----------+
|         common_name|                 scientific_name|taxonomy_id|   assembly|assembly_accession|              genebuild|class|order|family|species|maximum_lifespan|body_mass_g|metabolic_rate|temperature|gestation_days|female_maturity_days|male_maturity_days|weaning|litter_size|litters_per_year|inter_birth_interval|birth_weight_g|weaning_weight_g|adult_weight_g|growth_rate|
+--------------------+--------------------------------+-----------+-----------+------------------+-----------------------+-----+-----+------+-------+----------------+-----------+--------------+-----------+--------------+--------------------+---

In [5]:
/*
val ensemble_species = spark.readTSV("/data/ensembl/99/Species.csv", sep=",", header = true)
 .withColumnRenamed("Scientific Name", "scientific_name")
 .withColumnRenamed("Taxon ID", "taxon_id")
 .select("scientific_name", "taxon_id")

 ensemble_species.show(10,100)
*/

+--------------------------+--------+
|           scientific_name|taxon_id|
+--------------------------+--------+
|     Orycteropus afer afer| 1230840|
|    Chelonoidis abingdonii|  106734|
|Struthio camelus australis|  441894|
|        Gopherus agassizii|   38772|
|               Mus spretus|   10096|
|             Vicugna pacos|   30538|
|   Marmota marmota marmota|    9994|
|          Poecilia formosa|   48698|
|         Castor canadensis|   51338|
|         Bison bison bison|   43346|
+--------------------------+--------+
only showing top 10 rows



In [6]:
/*
val species = anage_animals.join(ensemble_species, "scientific_name")
  .withColumn("url", concat(lit("https://www.ensembl.org/"), $"genus", lit("_"), $"species"))  
  .sort($"class", $"maximum_lifespan".desc)
species.show()
*//*
val species = anage_animals.join(ensemble_species, "scientific_name")
  .withColumn("url", concat(lit("https://www.ensembl.org/"), $"Genus", lit("_"), $"Species"))
  .withColumnRenamed("tax_id", "taxon_id")
  .sort($"Class", $"Maximum longevity (yrs)".desc)
  .select($"taxon_id".as[String],
      $"scientific_name".as("scientific_name").as[String], 
      $"Common name".as("common_name").as[String],
      $"url".as[String],
  $"Class".as("class").as[String], 
  $"Maximum longevity (yrs)".as("lifespan").as[String],
  $"Body mass (g)".as("mass_g").as[String],
  $"Metabolic rate (W)".as("metabolic_rate"),
  $"Temperature (K)".as("temperature_k"),
  $"Specimen origin".as("specimen_origin").as[String],
  $"Sample size".as("sample_size").as[String],
  $"Data quality".as("data_quality").as[String]
  ).sort($"class", $"lifespan".desc)
species.show()*/

+---------------+------+-----+-----+------+-----+-------+-----------+----------------+-----------+--------------+-----------+------------------+--------------+--------------------+------------------+-------+-----------+----------------+--------------------+--------------+----------------+--------------+-----------+------+---------------+-----------+------------+------------+----------+----------+--------+---+
|scientific_name|phylum|class|order|family|genus|species|common_name|maximum_lifespan|body_mass_g|metabolic_rate|temperature|temperature_kelvin|gestation_days|female_maturity_days|male_maturity_days|weaning|litter_size|litters_per_year|inter_birth_interval|birth_weight_g|weaning_weight_g|adult_weight_g|growth_rate|source|specimen_origin|sample_size|data_quality|imr_per_year|mrdt_years|references|taxon_id|url|
+---------------+------+-----+-----+------+-----+-------+-----------+----------------+-----------+--------------+-----------+------------------+--------------+---------------

In [8]:
//species.distinct.writeParquet("/data/ensembl/99/website/species_lookup.parquet", true)
//species.distinct.writeTSV("/data/ensembl/99/website/species_lookup.tsv", true)

parts of /data/ensembl/99/website/species_lookup.parquet merged!


/data/ensembl/99/website/species_lookup.parquet

## Writing genes



In [10]:
val prefix_ens = "http://rdf.ebi.ac.uk/resource/ensembl/"
val prefix = "http://aging-research.group/resource/"
def ens(str: String) = "<"+prefix_ens+str.replace(" ", "_")+">"
def tax(str: String) = ens("taxon#"+str)
def pref(str: String) = "<" +prefix +str.replace(" ", "_")+">" 
def str(s: String) = s""""${s.replace("\"", "\\\"")}"""" 
    //+"^^<http://www.w3.org/2001/XMLSchema#string>"

In [11]:
def trip(s: String, p: String, o: String):List[(String, String, String, String)] = if(s==null || p == null || o == null ||
 s == "" ||  p == "" || o == "" || o=="\\N" || o.startsWith("\"\\N\"") || s == "\\N" || s == "\"\\N\"") Nil else List((s,p,o, "<" + "http://rdf.ebi.ac.uk/resource/ensembl/confidence/high" + "> ."))
def sp(taxon_id: java.lang.Integer, scientific_name: String)(p: String, o: Any) = if(taxon_id==null || o == null) Nil else {
      trip(tax(taxon_id.toString), pref(p), o.toString)++
        trip(pref(scientific_name),pref(p), o.toString) 
}

In [12]:
trs.writeTSV("/data/databases/graphdb/import/species.nt", false)

Error: not found: value trs (0)

## **Making a list of genes**<br>




In [14]:
val gene_members = spark.readTSV("/data/ensembl/99/compara/dump/gene_member.txt").toDF("gene_member_id","stable_id","version","source_name","taxon_id","genome_db_id","biotype_group","canonical_member_id",
"description","dnafrag_id","dnafrag_start","dnafrag_end","dnafrag_strand","display_label").select("stable_id", "display_label", "taxon_id", "biotype_group", "description").distinct()
gene_members.show(10)

+------------------+-------------+--------+-------------+--------------------+
|         stable_id|display_label|taxon_id|biotype_group|         description|
+------------------+-------------+--------+-------------+--------------------+
|ENSPPYG00000017407|           \N|    9601|       coding|                  \N|
|ENSPPYG00000017632|        ELMO1|    9601|       coding|engulfment and ce...|
|ENSPPYG00000017668|           \N|    9601|       coding|                  \N|
|ENSTNIG00000015267|        OTOL1|   99883|       coding|otolin 1 [Source:...|
|ENSTNIG00000007702|        mfsd1|   99883|       coding|major facilitator...|
|ENSPPYG00000018066|         WEE2|    9601|       coding|WEE2 oocyte meios...|
|ENSTNIG00000016189|      zbtb16b|   99883|       coding|zinc finger and B...|
|ENSTNIG00000016255|   zgc:152857|   99883|       coding|zgc:152857 [Sourc...|
|ENSPPYG00000000012|           \N|    9601|       coding|olfactory recepto...|
|ENSPPYG00000000045|         CNST|    9601|       co

In [15]:
val sm_species = species.withColumnRenamed("taxonomy_id", "taxon_id").select("scientific_name", "common_name", "taxon_id", "class")
val genes = gene_members.join(sm_species, "taxon_id").sort($"class", $"scientific_name", $"biotype_group", $"stable_id").distinct()
genes.show()

+--------+------------------+-------------+-------------+--------------------+--------------------+-------------+-----+
|taxon_id|         stable_id|display_label|biotype_group|         description|     scientific_name|  common_name|class|
+--------+------------------+-------------+-------------+--------------------+--------------------+-------------+-----+
|   80966|ENSAPOG00000000002|    zgc:63972|       coding|protein CutA homo...|Acanthochromis_po...|Spiny chromis| null|
|   80966|ENSAPOG00000000003|           \N|       coding|                  \N|Acanthochromis_po...|Spiny chromis| null|
|   80966|ENSAPOG00000000004|          rhd|       coding|ammonium transpor...|Acanthochromis_po...|Spiny chromis| null|
|   80966|ENSAPOG00000000005|           \N|       coding|protein bicaudal ...|Acanthochromis_po...|Spiny chromis| null|
|   80966|ENSAPOG00000000006|     ppp1r27b|       coding|protein phosphata...|Acanthochromis_po...|Spiny chromis| null|
|   80966|ENSAPOG00000000007|       capn

In [16]:
genes.writeParquet("/data/ensembl/99/website/genes_lookup.parquet", true)

parts of /data/ensembl/99/website/genes_lookup.parquet merged!


/data/ensembl/99/website/genes_lookup.parquet

In [17]:
genes.writeTSV("/data/ensembl/99/website/genes_lookup.tsv", true)

parts of /data/ensembl/99/website/genes_lookup.tsv merged!


/data/ensembl/99/website/genes_lookup.tsv

In [18]:
genes.count() //4383816

19358366